# Rule: **solve_sector_network**

**Outputs**

- results/networks/`base_s_{clusters}_{opts}_{sector_opts}_{horizon}.nc`

In [ ]:
######################################## Parameters

### Run
name = 'case_00_1H'
prefix = ''

### Network
clusters = 5
opts = ''
sector_opts = ''
horizon = '2030'

In [ ]:
##### Import packages
import pypsa
import pandas as pd
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import yaml
import os 
import sys
import numpy as np


##### Import local functions
sys.path.append(os.path.abspath(os.path.join('..')))
import functions as xp


##### Read params.yaml
with open('../params.yaml', 'r') as configfile:
    params = yaml.safe_load(configfile)


##### Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


##### Region files
file_regions_onshore = f'regions_onshore_base_s_{clusters}.geojson'
file_regions_offshore = f'regions_offshore_base_s_{clusters}.geojson'
path_regions = f'{params['rootpath']}/resources/{prefix}/{name}/'
gdf_regions_onshore = gpd.read_file(path_regions+file_regions_onshore)
gdf_regions_offshore = gpd.read_file(path_regions+file_regions_offshore)


##### NUTS files (provided by the user, used here to display results at NUTS level. The files must contain at least the columns 'NUTS_ID' and 'geometry')
file_NUTS2 = 'NUTS2_ES.geojson'
file_NUTS3 = 'NUTS3_ES.geojson'
path_NUTS = f'{params['rootpath']}/data_ES/nuts/'
gdf_NUTS2 = gpd.read_file(path_NUTS+file_NUTS2)
gdf_NUTS3 = gpd.read_file(path_NUTS+file_NUTS3)


## `base_s_{clusters}_{opts}_{sector_opts}_{horizon}.nc`

Load the network and show its components.

In [ ]:
file = f'base_s_{clusters}_{opts}_{sector_opts}_{horizon}.nc'
path = f'{params['rootpath']}/results/{prefix}/{name}/networks/'

n = pypsa.Network(path+file)

n

Plot the initial and optimal networks.

In [ ]:
#################### Parameters
line_widths_0 = 1*n.lines.s_nom / 1e3
link_widths_0 = 1*n.links.p_nom / 1e3

line_widths_1 = 1*n.lines.s_nom_opt / 1e3
link_widths_1 = 1*n.links.p_nom_opt / 1e3


#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, axes = plt.subplots(1, 2, figsize=fig_size, subplot_kw={'projection': crs})


##### Initial network
ax0 = axes[0]

### Add network
n.plot(ax=ax0, 
       line_widths=line_widths_0, 
       link_widths=link_widths_0, 
       bus_sizes=params['bus_sizes'], 
       bus_colors=params['bus_colors'], 
       boundaries=params['boundaries_offshore'])

### Add regions_onshore
xp.map_add_region(ax0, gdf_regions_onshore, params['map_add_region'])

### Add regions_offshore
xp.map_add_region(ax0, gdf_regions_offshore, params['map_add_region'], is_offshore=True)

### Add map features
xp.map_add_features(ax0, params['map_add_features'])



##### Optimal network
ax1 = axes[1]

### Add network
n.plot(ax=ax1, 
       line_widths=line_widths_1, 
       link_widths=link_widths_1, 
       bus_sizes=params['bus_sizes'], 
       bus_colors=params['bus_colors'], 
       boundaries=params['boundaries_offshore'])

### Add regions_onshore
xp.map_add_region(ax1, gdf_regions_onshore, params['map_add_region'])

### Add regions_offshore
xp.map_add_region(ax1, gdf_regions_offshore, params['map_add_region'], is_offshore=True)

### Add map features
xp.map_add_features(ax1, params['map_add_features'])

Plot the increase in line capacities.

### Variable: `n.generators`

Place `n.generators` in a dataFrame and show its content.

In [ ]:
gg = n.generators

gg.head()

#### Summary

What is the aggregated capacity, initial and optimal, per carrier? 

In [ ]:
gg.groupby('carrier').agg(
    Total_capacity=pd.NamedAgg(column='p_nom', aggfunc='sum'),
    Optimal_capacity=pd.NamedAgg(column='p_nom_opt', aggfunc='sum'),
)

#### Maps

Plot a map showing a particular feature of a generation carrier at each region.

Then, show another map with the same information aggregated to a certain NUTS level (weighted with ovelap area).

In [ ]:
#################### Parameters

### Select carrier
carrier = 'solar'

### Select feature (uncomment one of the following):
# feature = 'area' 
# feature = 'p_nom'
# feature = 'p_nom_density'
# feature = 'p_nom_max'
# feature = 'p_nom_max_density'
#feature = 'p_nom_max_ratio'
feature = 'p_nom_opt'
# feature = 'p_nom_opt_density'
# feature = 'p_nom_opt_max_ratio'


#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define gdf_regions
if 'off' in carrier:
    gdf_regions = gdf_regions_offshore
    is_offshore = True
else:
    gdf_regions = gdf_regions_onshore
    is_offshore = False


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_generators(carrier, n, feature, ax, gdf_regions, params['map_network_generators'], params_local)

In [ ]:
#################### Parameters

### Select NUTS level
NUTS_level = 3



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



if 'off' in carrier:
    print('Aggregation at NUTS level for offshore is not possible')
else:
    #################### Figure
    fig_size = [12,6]
    crs = ccrs.PlateCarree()

    fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


    ### Define gdf_regions
    gdf_regions = gdf_regions_onshore
    is_offshore = False


    ### Define NUTS file
    if NUTS_level==2:
        gdf_NUTS = gdf_NUTS2
    if NUTS_level==3:
        gdf_NUTS = gdf_NUTS3    


    ### Add map features
    xp.map_add_features(ax, params['map_add_features'])

    ### Add network feature aggregated at NUTS regions
    xp.map_NUTS_generators(carrier, n, f'{feature}_NUTS', ax, gdf_regions, gdf_NUTS, params['map_NUTS_generators'], params_local)


#### Costs

What is the capital cost of generators?

In [ ]:
#################### Parameters

### Select carriers
carrier_list = ['onwind', 'solar', 'offwind-float']



#################### Figure
fig_size = [10,5]
fig, ax = plt.subplots(1,1,figsize=fig_size)

# Give a message with the carriers not considered
carrier_all = gg['carrier'].unique().tolist()
carrier_excluded = [carrier for carrier in carrier_all if carrier not in carrier_list]
print(f'Carriers ommitted: {carrier_excluded}')

# Define bins
minimo = 0 # 0.95 * gg.loc[ gg['carrier'].isin(carrier_list), 'capital_cost'].min()
maximo = 1.05 * gg.loc[ gg['carrier'].isin(carrier_list), 'capital_cost'].max()
ancho = 1000

bins = np.arange(minimo, maximo, 1000)

# Define colors
tech_colors = n.carriers['color']


for carrier in carrier_list:

    ##### Filter the carrier
    df = gg[gg['carrier']==carrier]


    ##### Only single cost for the carrier
    if df['capital_cost'].round(2).nunique()==1:

        valor = df['capital_cost'].unique()[0]

        ax.axvline(x=valor, label=carrier, color = tech_colors[carrier])

        print(f'Capital cost for {carrier} is: {valor:.2f} EUR/MW·year')

    ##### Different costs for the carrier
    else:
        plt.hist(df['capital_cost'], bins=bins, 
                 edgecolor='none', color = tech_colors[carrier],
                 label=carrier, alpha=1)
        
        valor = df['capital_cost'].mean()
        print(f'Average capital cost for {carrier} is: {valor:.2f} EUR/MW·year')


ax.set_title('capital cost')
ax.set_xlabel('EUR/(MW·year)')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.5)

### Variable: `n.global_constraints`

Place `n.global_constraints` in a dataFrame and show its content.

In [ ]:
gc = n.global_constraints

gc

### Variable: `n.lines`

Place `n.lines` in a dataFrame and show its content.

In [ ]:
ln = n.lines

ln.head()

How is the distribution of line lengths?

In [ ]:
#################### Parameters
bins = 50



#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


ax.hist(ln['length'], bins=bins, edgecolor='black')

ax.set_xlabel('km')
ax.grid(True, linestyle='--', alpha=0.5)

How is the relationship between line capital costs and line length?

In [ ]:
#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


ln.plot(ax=ax, kind='scatter', x='length', y='capital_cost')

ax.set_xlabel('km')
ax.set_ylabel('EUR/MW')
ax.grid(True, linestyle='--', alpha=0.5)


ratio = ln['capital_cost']/ln['length']

print(f'The ratio values of capital cost vs. length  are {ratio.round(2).unique()} EUR/(MW·km)')

### Variable: `n.links`

Place `n.links` in a dataFrame and show its content.

In [ ]:
lk = n.links

lk.head()

Place DC links in a dataFrame and show its content.

In [ ]:
lk_DC = lk.loc[lk['carrier']=='DC']

lk_DC.head()

How is the distribution of DC link lengths?

In [ ]:
#################### Parameters
bins = 50



#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


ax.hist(lk_DC['length'], bins=bins, edgecolor='black')

ax.set_xlabel('km')
ax.grid(True, linestyle='--', alpha=0.5)

How is the relationship between DC link capital costs and link length?

In [ ]:
#################### Figure
fig_size = [10,4]
fig, ax = plt.subplots(figsize=fig_size)


lk_DC.plot(ax=ax, kind='scatter', x='length', y='capital_cost')

ax.set_xlabel('km')
ax.set_ylabel('EUR')
ax.grid(True, linestyle='--', alpha=0.5)


ratio = lk_DC['capital_cost']/lk_DC['length']

print(f'The ratio values of capital cost vs. length  are {ratio.round(2).unique()} EUR/km')

**NOTE**: Links represent different elements in the network. 
 For example, for CCGT technology, the link represents the conversion from gas to electricity, and the associated capacity refers to the input node. However, in general we are interested in the CCGT capacity related to the electrical power output (output node).
 For this reason, the capacity associated to the link needs to be multiplied by the efficiency for `CCGT`, `OCGT`, `H2 Fuel Cell` and `battery discharger` to properly the reflect output power.

In the following, **"_e"** is added to express "electric" (power output, the efficiency is included).

#### Maps

Plot a map showing a particular feature of a link carrier at each region.

Then, show another map with the same information aggregated to a certain NUTS level (weighted with ovelap area).

In [ ]:
#################### Parameters

### Select carrier (uncomment one of the following)
carrier = 'CCGT'
# carrier = 'OCGT'
# carrier = 'H2 Electrolysis'
# carrier = 'H2 Fuel Cell'
# carrier = 'battery charger'
# carrier = 'battery discharger'

### Select feature (uncomment one of the following):
# feature = 'area' 
# feature = 'p_nom_e'
feature = 'p_nom_e_opt'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_links(carrier, n, feature, ax, gdf_regions_onshore, params['map_network_links'], params_local)

In [ ]:
#################### Parameters

### Select NUTS level
NUTS_level = 3



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define NUTS file
if NUTS_level==2:
    gdf_NUTS = gdf_NUTS2
if NUTS_level==3:
    gdf_NUTS = gdf_NUTS3    


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature aggregated at NUTS regions
xp.map_NUTS_links(carrier, n, f'{feature}_NUTS', ax, gdf_regions_onshore, gdf_NUTS, params['map_NUTS_links'], params_local)

### Variable: `n.stores`

Place `n.stores` in a dataFrame and show its content.

In [ ]:
st = n.stores

st.head()

#### Maps

Plot a map showing a particular feature of a store carrier at each region.

Then, show another map with the same information aggregated to a certain NUTS level (weighted with ovelap area).

In [ ]:
#################### Parameters

### Select carrier (uncomment one of the following):
carrier = 'battery'
# carrier = 'H2 Store'

### Select feature (uncomment one of the following):
feature = 'e_nom_opt'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_stores(carrier, n, feature, ax, gdf_regions_onshore, params['map_network_stores'], params_local)

In [ ]:
#################### Parameters

### Select NUTS level
NUTS_level = 3



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define NUTS file
if NUTS_level==2:
    gdf_NUTS = gdf_NUTS2
if NUTS_level==3:
    gdf_NUTS = gdf_NUTS3    


### Add regions
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'], is_offshore=is_offshore)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature aggregated at NUTS regions
xp.map_NUTS_stores(carrier, n, f'{feature}_NUTS', ax, gdf_regions_onshore, gdf_NUTS, params['map_NUTS_stores'], params_local)

### Variable: `n.storage_units`

Place `n.storage_units` in a dataFrame and show its content.

In [ ]:
su = n.storage_units

su.head()

#### Summary

What is the aggregated capacity per carrier? 

How many buses have storage units for each carrier? How many of them have zero capacity?

In [ ]:
su.groupby('carrier').agg(
    Total_capacity=pd.NamedAgg(column='p_nom', aggfunc='sum'),
    Buses=pd.NamedAgg(column='p_nom', aggfunc='size'),
    Buses_zero_capacity=pd.NamedAgg(column='p_nom', aggfunc=lambda x: (x == 0).sum()),
    Buses_non_zero_capacity=pd.NamedAgg(column='p_nom', aggfunc=lambda x: (x != 0).sum()),
)

#### Maps

Plot a map showing a particular feature of a storage unit carrier at each region.

Then, show another map with the same information aggregated to a certain NUTS level (weighted with ovelap area).

In [ ]:
#################### Parameters

### Select carrier
carrier = 'PHS'

### Select feature (uncomment one of the following):
# feature = 'area' 
# feature = 'p_nom'
# feature = 'p_nom_density'
feature = 'max_hours'



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature at regions
xp.map_network_storage_units(carrier, n, feature, ax, gdf_regions_onshore, params['map_network_storage_units'], params_local)

In [ ]:
#################### Parameters

### Select NUTS level
NUTS_level = 2



#################### Local params
params_local = {}
params_local['vmin'] = ''   # Leave empty for automatic value 
params_local['vmax'] = ''   # Leave empty for automatic value



#################### Figure
fig_size = [12,6]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Define NUTS file
if NUTS_level==2:
    gdf_NUTS = gdf_NUTS2
if NUTS_level==3:
    gdf_NUTS = gdf_NUTS3    


### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Add network feature aggregated at NUTS regions
xp.map_NUTS_storage_units(carrier, n, f'{feature}_NUTS', ax, gdf_regions, gdf_NUTS, params['map_NUTS_storage_units'], params_local)

#### Maximum hours

What is the relation between installed capacity and max. hours, for each carrier?

In [ ]:
#################### Figure
fig_size = [8,4]
fig, ax = plt.subplots(figsize=fig_size)

tech_colors = n.carriers['color']

for carrier, group in su.groupby('carrier'):
    ax.scatter(group['p_nom'], group['max_hours'], label=carrier, color=tech_colors[carrier], alpha=0.7)

ax.set_xlabel('Installed capacity [MW]')
ax.set_ylabel('Max. hours')
ax.set_title('Storage units')
ax.grid(True, linestyle='--', alpha=0.5)
ax.legend()

### Analysis: Capacity expansion

Plot initial and optimal capacities.

**Note:** Capacity associated to links `CCGT`, `OCGT`, `H2 Fuel Cell` and `battery discharger` have been multiplied by efficiencies to consider output power.

In [ ]:
#################### Parameters



#################### Get summary

df_capacities = df_network_capacities(n)



#################### Figure
fig_size = [10,6]



### Analysis: Grid expansion

If enabled, optimation includes grid expansion for lines and links.

Where was grid expansion required?

Plot the network showing the increase of line capacities (if any).

In [ ]:
#################### Parameters
line_widths = 1*(n.lines.s_nom_opt-n.lines.s_nom) / 1e3
link_widths = 1*(n.links.p_nom_opt-n.links.p_nom) / 1e3



#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add network
n.plot(ax=ax, 
       line_widths=line_widths, 
       link_widths=link_widths, 
       bus_sizes=params['bus_sizes'], 
       bus_colors=params['bus_colors'], 
       boundaries=params['boundaries_offshore'])

### Add regions_onshore
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add regions_offshore
xp.map_add_region(ax, gdf_regions_offshore, params['map_add_region'], is_offshore=True)

### Add map features
xp.map_add_features(ax, params['map_add_features'])

### Analysis: Grid congestion

Where are the bottlenecks in the grid from the optimal dispatch?

Plot the network showing the level of congestion.

In [ ]:
#################### Parameters

### Select criterion
# cong_criterion = ('mean', 'na')
cong_criterion = ('quantile', 0.9)

### Define line and link widths
line_widths = 1*(n.lines.s_nom_opt) / 1e3
link_widths = 0*(n.links.p_nom_opt) / 1e3
link_widths.loc[n.links.index.str.contains('ic')] = 1 * n.links.loc[n.links.index.str.contains('ic'), 'p_nom_opt'] / 1e3
link_widths.loc[n.links.index.str.contains('relation')] = 1 * n.links.loc[n.links.index.str.contains('relation'), 'p_nom_opt'] / 1e3



#################### This is a patch for considering ES interconnections

# for interconnections
for interconnection in ['FR0', 'FR1', 'PT0', 'PT1']:
    try:
        join_power = lkt_p0[f'ES {interconnection} ic'] - lkt_p0[f'ES {interconnection} ic-reversed']
        lkt_p0[f'ES {interconnection} ic'] = join_power
        lkt_p0[f'ES {interconnection} ic-reversed'] = join_power
    except KeyError:
        join_power = 0
        
    

# for link with Balearic islands
for interconnection in n.links.loc[n.links.index.str.contains('relation')].index:
    if 'reversed' not in interconnection:
        try:
            join_power = lkt_p0[interconnection] - lkt_p0[f'{interconnection}-reversed']
            lkt_p0[interconnection] = join_power
            lkt_p0[f'{interconnection}-reversed'] = join_power
        except KeyError:
            join_power = 0




#################### Get the line_colors

if cong_criterion[0] == 'mean':
    ln_result_criterion = lnt_p0.abs().mean() / n.lines['s_nom']
    lk_result_criterion = lkt_p0.abs().mean() / n.links['p_nom']
    title_cb = 'Mean of line CF'
if cong_criterion[0] == 'quantile':
    ln_result_criterion = lnt_p0.abs().quantile(cong_criterion[1]) / n.lines['s_nom']
    lk_result_criterion = lkt_p0.abs().quantile(cong_criterion[1]) / n.links['p_nom']
    title_cb = f'Q{cong_criterion[1]} of line CF'

norm = mcolors.Normalize(vmin=0, vmax=0.7)
cmap = plt.cm.rainbow  # You can choose any colormap you like

ln_colors = [cmap(norm(p)) for p in ln_result_criterion]
lk_colors = [cmap(norm(p)) for p in lk_result_criterion]



#################### Figure
fig_size = [12,12]
crs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': crs})


### Add network
n.plot(ax=ax, 
       line_widths=line_widths, 
       link_widths=link_widths, 
       line_colors=ln_colors, 
       link_colors=lk_colors,
       bus_sizes=params['bus_sizes'], 
       bus_colors=params['bus_colors'], 
       boundaries=params['boundaries_offshore']
       )


# Create a ScalarMappable for the colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Required for compatibility with colorbar creation
# Add the colorbar to the figure
cbar = fig.colorbar(sm, ax=ax, orientation='vertical', shrink=0.55, pad=0.02)
cbar.set_label(title_cb, fontsize=12)


### Add regions_onshore
xp.map_add_region(ax, gdf_regions_onshore, params['map_add_region'])

### Add map features
xp.map_add_features(ax, params['map_add_features'])


